## Cleaning and Combining of addresses

### Preparation & Data import

In [22]:
import pandas as pd
import re as re
import numpy as np

In [23]:
dateparser = lambda x: pd.datetime.strptime(x, '%Y%m%d')

dat = pd.read_csv('Bucher17.csv', sep=';', date_parser=dateparser, parse_dates=['Datum'])
dat.head()

,Datum,Buchungsnummer,Kundennummer,MIETE,Hausnation,Name1,Name2,Adresse1,Adresse2,LKZ,PLZ,Ort,Buchungskanal,Direkt_Büro
0,2016-07-05,100003,3355207,1246,DK,Heiner Warzecha,NaN,Laubstr. 24,NaN,D,33607,Bielefeld,AS400,Dir
1,2016-07-05,100004,3788878,1506,DK,"Knöchelmann, Matthias",NaN,Bahnhofstr.48 A,NaN,D,25560,Puls,AS400,Dir
2,2016-07-05,100005,3788879,1656,DK,"Lippert, Yvonne",NaN,Eichenkamp 7,NaN,D,21447,Handorf,AS400,Bur
3,2016-07-05,100010,3665618,1966,D,"Lindenblatt, Merle",NaN,Wiehenweg 135,NaN,D,32312,Lübbecke,AS400,Bur
4,2016-07-05,100011,3665618,2886,D,"Lindenblatt, Merle",NaN,Wiehenweg 135,NaN,D,32312,Lübbecke,AS400,Bur


In [25]:
sum(dat.duplicated())

137

### Functions for cleaning data

#### General Cleaning: Umlaute, leading, trailing spaces, Sonderzeichen, lower case

In [3]:
def clean_string(string):
    
    string = str(string)
    if string:

        # lower case, strip spaces
        string = string.lower().strip()

        # replace more than oone middle spaces
        string = re.sub(pattern=' {2,100}',repl=' ', string=string)

        # replacementes: Umlaute, einige Sonderzeichen
        string = string.replace('ä','ae')
        string = string.replace('ë','e')
        string = string.replace('ï','i')
        string = string.replace('ö','oe')
        string = string.replace('ü','ue')

        string = string.replace('ß','ss')

        string = string.replace('&','und')

        string = string.replace('à','a')
        string = string.replace('è','e')
        string = string.replace('ì','i')
        string = string.replace('ò','o')
        string = string.replace('ù','u')

        string = string.replace('á','a')
        string = string.replace('é','e')
        string = string.replace('í','i')
        string = string.replace('ó','o')
        string = string.replace('ú','u')

        string = string.replace('â','a')
        string = string.replace('ê','e')
        string = string.replace('î','i')
        string = string.replace('ô','o')
        string = string.replace('û','u')

        string = string.replace('å','a')
        string = string.replace('î','i')
        string = string.replace('ô','o')
        string = string.replace('û','u')

        string = string.replace('ñ','n')
        string = string.replace('ã','a')
        string = string.replace('ñ','n')
        string = string.replace('õ','o')

        string = string.replace('å','a')
        string = string.replace('ø','o')
    
    return(string)

In [4]:
def delete_spaces(string):
    return ''.join(string.split(' '))
    

#### Handling Name1 and Name2

In [5]:
# remove Herr, Hr., Dr. usw
def remove_title(string):
    string = string.replace('herr','')
    string = string.replace('hr.','')
    string = string.replace('frau','')
    string = string.replace('fr.','')
    string = string.replace('doktor','')
    string = string.replace('dr.','')
    string = string.replace('professor','')
    string = string.replace('prof.','')
    string = string.replace('familie','')
    
    string = string.strip()
    
    return(string)

In [6]:
# Vor- und Nachname

gewerblich=['stiftung','verein','club','hilfe','anstalt',
            'service','gmbh','institut','sozial','gruppe',
            'centrum','gruppe','zentrum','gemeinde','e.v.','e. v.',
           'ggmbh', 'spitzke']

def name_handling_firstname(name):
    
    # und, u
    name = name.replace(' und ',' u. ')
    name = name.replace(' u ',' u. ')
    # replace more than one middle spaces
    name = re.sub(pattern=' {2,100}',repl=' ', string=name)
    
    # gewerbliche Adressen
    for word in gewerblich:
        if word in name:
            print(name)
            
            return np.nan
    
    # andere namen werden an ' ', '/' , ',' getrennt 
    name = str(name)
    name =  name.lower()
    
    komma1_split = name.split(', ')
    komma2_split = name.split(',')

    slash_split = name.split('/')
    space_split = name.split(' ')
    
    if len(komma1_split)>1:
        return ' '.join(komma1_split[1:])
    if len(komma2_split)>1:
        return ' '.join(komma2_split[1:])
    if len(slash_split)>1:
        return ' '.join(slash_split[1:])
         
    return ' '.join(space_split[:-1])

def name_handling_lastname(name):
    
    # und, u
    name = name.replace(' und ',' u. ')
    name = name.replace(' u ',' u. ')
    # replace more than one middle spaces
    name = re.sub(pattern=' {2,100}',repl=' ', string=name)
    
    # gewerbliche Adressen
    for word in gewerblich:
        if word in name:
            print(name)
            
            return name
    
    komma1_split = name.split(', ')
    komma2_split = name.split(',')
    slash_split = name.split('/')
    doublespace_split = name.split('  ')
    space_split = name.split(' ')
    
    if len(komma1_split)>1:
        return komma1_split[0]
    if len(komma2_split)>1:
        return komma2_split[0]
    if len(slash_split)>1:
        return slash_split[0]
    if len(doublespace_split)>1:
        return doublespace_split[0]
        
    return space_split[-1]

#### Straßen, Hausnummern, Orte, PLZ

In [7]:
# Abkürzungen, '-', '/'

def clean_streetnames_loaction(streetname):
    streetname = streetname.replace('-', ' ') 
    streetname = streetname.replace('/', ' ') 
    
    
    streetname = streetname.replace('strasse', 'str.')
    streetname = streetname.replace('str ', 'str. ')
    streetname = streetname.replace('platz', 'pl.')
    streetname = streetname.replace('pl ', 'pl. ')
    
    streetname = streetname.replace('nummer', 'nr.')
    
    streetname = streetname.replace(' an der ', ' a.d. ') 
    streetname = streetname.replace(' an den ', ' a.d. ') 
    streetname = streetname.replace(' an dem', ' a.d. ') 
    streetname = streetname.replace(' auf der ', ' a.d. ') 
    streetname = streetname.replace(' auf dem ', ' a.d. ') 
    streetname = streetname.replace('a. d.', 'a.d.')

    streetname = streetname.replace(' in der ', ' i.d. ') 
    streetname = streetname.replace(' in dem ', ' i.d. ') 
    streetname = streetname.replace(' in den ', ' i.d. ') 
    streetname = streetname.replace(' im ', ' i.d. ') 
    streetname = streetname.replace('i. d.', 'i.d.')
    
    
    streetname = streetname.replace('am',' ')    
    streetname = streetname.replace('a.d.', ' ')
    streetname = streetname.replace('i.d.', ' ')
    
    streetname = streetname.replace(' und ',' u. ')
    streetname = streetname.replace(' u ',' u. ')
    
    streetname = streetname.replace('obere ', 'ob. ')
    streetname = streetname.replace('oberer ', 'ob. ')
    streetname = streetname.replace('oberes ', 'ob. ')
    
    streetname = streetname.replace('unter ', 'u. ')
    streetname = streetname.replace('unterer ', 'u. ')
    streetname = streetname.replace('unteres ', 'u. ')
    
    streetname = streetname.replace('ueber ', 'ue. ')
    streetname = streetname.replace('von ', 'v. ')
    streetname = streetname.replace('vor ', 'v. ')
    streetname = streetname.replace('vom ', 'v. ')
    streetname = streetname.replace('zum ', 'z. ')
    streetname = streetname.replace('zur ', 'z. ')
    
    streetname = streetname.replace(' der ', ' d. ')
    streetname = streetname.replace(' das ', ' d. ')
    streetname = streetname.replace(' die ', ' d. ')
    streetname = streetname.replace(' dem ', ' d. ')
    streetname = streetname.replace(' den ', ' d. ')
        
    streetname = streetname.replace('buergermeister', 'bgm.')
    streetname = streetname.replace('roemisch', 'roem.')
    streetname = streetname.replace('bruecke', 'br.')
    streetname = streetname.replace('br ', 'br. ')
    streetname = streetname.replace(' ortsteil ', 'ot. ')
    streetname = streetname.replace(' ot ', ' ot. ')
    
    
    
    # replace more than one middle spaces
    streetname = re.sub(pattern=' {2,100}',repl=' ', string=streetname)
    
    return streetname    

In [8]:
def extract_housenumber(streetname):
    # Hausnr. Typ 1212 1212 (auch 123-123 und 123/123), 1212a3232
    hn=re.search('[0-9]{1,9}([ ]|[a-z])[0-9]{1,9}',streetname)
    if hn != None:
        return hn[0]
    
    # Hausnr. Typ 123a, 123 a
    hn=re.search('[0-9]{1,9}(( [a-z])|[a-z])',streetname)
    if hn != None:
        # strip away middle spaces
        return re.sub(pattern=' ',repl='', string=hn[0])
    
    # Hausnr. Typ 123
    hn=re.search('[0-9]{1,9}',streetname)
    if hn != None:
        return hn[0]
    
    return np.nan
    
    
def delete_housenumber(streetname):
    # Hausnr. Typ 1212 1212 (auch 123-123 und 123/123), 1212a3232
    hn=re.search('[0-9]{1,9}([ ]|[a-z])[0-9]{1,9}',streetname)
    if hn != None:
        return streetname[:hn.start()].strip()
    
    # Hausnr. Typ 123a, 123 a
    hn=re.search('[0-9]{1,9}(( [a-z])|[a-z])',streetname)
    if hn != None:
        return streetname[:hn.start()].strip()
    
    # Hausnr. Typ 123
    hn=re.search('[0-9]{1,9}',streetname)
    if hn != None:
        return streetname[:hn.start()].strip()
    
    return streetname
    

In [9]:
def delete_leading_zeros(plz):
    plz = str(plz)
    leading_zero = re.search('^0',plz)
    if leading_zero != None:
        return plz[leading_zero.end():]
    return plz

### Apply functions to Buchungen

##### General cleaning

In [10]:
dat['Name1_repl'] = dat['Name1'].apply(clean_string)
dat['Name2_repl'] = dat['Name2'].apply(clean_string)
dat['Adresse1_repl'] = dat['Adresse1'].apply(clean_string)
dat['Adresse2_repl'] = dat['Adresse2'].apply(clean_string)
dat['Ort_repl'] = dat['Ort'].apply(clean_string)

##### First, last name

In [11]:
dat['Name1_repl'] = dat['Name1_repl'].apply(remove_title)
dat['Name2_repl'] = dat['Name2_repl'].apply(remove_title)

dat['vorname_clean'] = dat['Name1_repl'].apply(name_handling_firstname)
dat['nachname_clean'] = dat['Name1_repl'].apply(name_handling_lastname)

lebenshilfe remscheid e.v
lebenshilfe wolfsburg gg
lebenshilfe wolfsburg gg
lebenshilfe wesermarsch annette meyer
lebenshilfe wesermarsch annette meyer
ipz gmbh
srs e.v. z.hd. sven jaeger
ervo gmbh
lebenshilfe-werk kreis waldeck-
lebenshilfe wesermarsch
lebenshilfe wesermarsch
lebenshilfe wesermarsch
jugendhilfe wuemmetal gmbh stefanie kroh
mosaik werkstaetten ggmbh
lebenshilfe bremervoerde/zeven
magdeburg universitaet sportcentrum
magdeburg universitaet sportcentrum
magdeburg universitaet sportcentrum
zentrum fuer jugendliche diabetiker
flexible jugendhilfe huelquist
flexible jugendhilfe huelquist
spastikerhilfe berlin eg
spastikerhilfe berlin eg
caritas mecklenburg e.v.
die wattenbeker gmbh
caritas zentrum
lebenshilfe bremervoerde/zeven
lebenshilfe bremervoerde-zeven
networkteam gmbh
lebenshilfe wohnen u. leben g
seniorenzentrum friedensheim
paritaetische lebenshilfe
paritaetische lebenshilfe
mauser einrichtungs.gmbhundco.kg
mauser einrichtungs gmbhundco kg
mauser einrichtungs gmbhun

erdwerke gmbh
emk gmbh
tankschutz vier-mann gmbh
firma/spitzke se danmark
firma/spitzke se danmark
strabag rail gmbh/bereich gleisbaumaschi
kinetics germany gmbh
kobra gmbh m rehbein
guenter meier innenausbau gmbh
kinetics germany gmbh
gebrueder rueckert gmbh und
kinetics germany gmbh
slg/spitzke logistik gmb
spitzke/se danmark
opus marine gmbh
riedel filtertechnik gmbh
riedel filtertechnik gmbh
firma/spitzke se danmark
firma/spitzke se danmark
gruppe spaetlich
gruppe spaetlich
gruppe spaetlich
gruppe spaetlich
gruppe spaetlich
gruppe spaetlich
spitzke/se danmark/
slg/spitzke logistik gmb
slg/spitzke logistik gmb
spitzke/se danmark/
spitzke/se danmark/
diak. behindertenhilfe lilienthal
spitzke se/danmark
spitzke/se danmark
spitzke/se danmark
spitzke/se danmark
spitzke/se danmark
firma/spitzke se danmark
spitzke/se danmark
spitzke/se danmark
minimax e.i.s. gmbh
firma/spitzke se danmark
firma/spitzke se danmark
spitzke/se danmark
spitzke/se danmark
spitzke/se danmark
spitzke/se danmark
s

seniorenzentrum nora amal care gmbh
gruppe jobi
gruppe jobi
gruppe jobi
gruppe jobi
gruppe jobi
gruppe jobi
gruppe jobi
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
gruppe weske
slg spitzke logistik gmb
slg spitzke logistik gmb
slg spitzke logistik gmb
kinetics germany gmbh
moeller feuerfest u. technik gmbh
moeller feuerfest u. technik gmbh
moeller feuerfest u. technik gmbh
moeller feuerfest u. technik gmbh
wassermann kabeltechnik gmbh
ahb leipzig gmbh
hephata hessisches diakonie zentrum e.v.
kroatische kulturgem. hamburg e.v.
rdj ggmbh
schumann tank u. stahlbau gmbh
hof althing gem. gmbh
network movie film gmbh
network movie film gmbh
network movie film gmbh
network movie film gmbh
network movie film gmbh
kinetics germany gmbh
kinetics germany gmbh
kinetics germany gmbh
gruppe langnickel
gruppe langnickel
gruppe langnickel
gruppe langnickel
gruppe langnickel
gruppe langnickel

In [12]:
# check first names by comparison with first name list from the internet
vornamen_list = pd.read_csv('../vornamen.csv',sep=';',header=None,encoding='latin-1')
vornamen_list = vornamen_list[0].tolist()
vornamen_list = [clean_string(x) for x in vornamen_list]

def check_vorname(name):
    if not any(vorname in name for vorname in vornamen_list):
        print(name)
        
dat['vorname_clean'] = pd.Series([str(x) for x in dat['vorname_clean'].tolist()])                    
dat['vorname_clean'].apply(check_vorname)


jewgeni
21184
haye
haye
haye
haye
asmus

jirko
zahnaerzte
pe
36964
ocka
baersch

kjer
t

geis
karsta
iren

heimstaette
heimstaette
heimstaette
heimstaette
heimstaette
heimstaette
fcm trav

ke

mooser
mooser
ke

ke
bigit
voss

oaf

h
wfbm
fra
wamser

schostak
bauer
birga















jærg

s.




freimut
onne
wietze
swcibba
kta
mathas

ren'e
yasmen
c.

egmont
a.
ke













turat
asmuth


ke

caesrin
frieser

a

brux
s.
b
hornig

n

jes
raija

per

stein-spitczok
enwer
schmitz

ritva











gazi
asmuth

gosse
recai
quynh ngoc
u











maique dos
ke




m.
rawi
k.
renz
soern
rajko

uve

u

biskupek
hei

osas

s




bienja

tueshaus

bintu
ke


steen









m


gessner







s.
inske

sansra

ciera
aykut

soehnke
ke
maass

a.w.
a.w.



t.
ke
tasso tasso





d

opitz


n


j
nitsch
mirca


sysa
birge





eyk


corn
fa.


braun
ke
ke
astrit

catja
anjs
ke

khairy



grasshoff

ke

eiko

howe

s.





haus
pirko
pirko
pirko
pirko
pirko

schmidt




ruett



ires


aid
maic

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
54713    None
54714    None
54715    None
54716    None
54717    None
54718    None
54719    None
54720    None
54721    None
54722    None
54723    None
54724    None
54725    None
54726    None
54727    None
54728    None
54729    None
54730    None
54731    None
54732    None
54733    None
54734    None
54735    None
54736    None
54737    None
54738    None
54739    None
54740    None
54741    None
54742    None
Name: vorname_clean, Length: 54743, dtype: object

In [13]:
dat[dat['vorname_clean']=='pirko']

,Datum,Buchungsnummer,Kundennummer,MIETE,Hausnation,Name1,Name2,Adresse1,Adresse2,LKZ,...,Ort,Buchungskanal,Direkt_Büro,Name1_repl,Name2_repl,Adresse1_repl,Adresse2_repl,Ort_repl,vorname_clean,nachname_clean
23996,2017-02-09,164478,3994685,515,DK,Pirko Post,NaN,Mohnstieg 4,NaN,D,...,Kiel,AS400,Bur,pirko post,nan,mohnstieg 4,nan,kiel,pirko,post
23997,2017-02-09,164478,3994685,515,DK,Pirko Post,NaN,Mohnstieg 4,NaN,D,...,Kiel,AS400,Bur,pirko post,nan,mohnstieg 4,nan,kiel,pirko,post
23998,2017-02-09,164478,3994685,515,DK,Pirko Post,NaN,Mohnstieg 4,NaN,D,...,Kiel,AS400,Bur,pirko post,nan,mohnstieg 4,nan,kiel,pirko,post
23999,2017-02-09,164478,3994685,515,DK,Pirko Post,NaN,Mohnstieg 4,NaN,D,...,Kiel,AS400,Bur,pirko post,nan,mohnstieg 4,nan,kiel,pirko,post
24000,2017-02-09,164480,3994685,671,DK,Pirko Post,NaN,Mohnstieg 4,NaN,D,...,Kiel,AS400,Bur,pirko post,nan,mohnstieg 4,nan,kiel,pirko,post


##### Streets, house numbers locations

In [14]:
# copying to separate column  and cleaning of first address field 
# (contains most information) and location field 
dat['strasse_clean']=dat['Adresse1_repl'].apply(clean_streetnames_loaction)
dat['hausnr_clean']=dat['Adresse1_repl'].apply(clean_streetnames_loaction)
dat['ort_clean']=dat['Ort_repl'].apply(clean_streetnames_loaction)
dat.ort_clean = dat.ort_clean.apply(delete_spaces)

# extract street and housenumber from first address field
dat['strasse_clean'] = dat['strasse_clean'].apply(delete_housenumber)
dat.strasse_clean = dat.strasse_clean.apply(delete_spaces)
dat['hausnr_clean'] = dat['hausnr_clean'].apply(extract_housenumber)

# delete leading zeros in post codes
dat['plz_clean'] = dat['PLZ'].apply(delete_leading_zeros)

In [15]:
# check housenumbers
dat.groupby('hausnr_clean').size().sort_values(ascending=False)[:10]

hausnr_clean
1     1809
3     1781
2     1749
4     1653
5     1601
7     1561
6     1551
8     1541
10    1349
9     1321
dtype: int64

### Make new key + save

In [16]:
dat['newKey'] = dat['nachname_clean']+' '+dat['strasse_clean']+dat['hausnr_clean']+' '+dat['plz_clean']+dat['ort_clean']

In [17]:
dat.head()

,Datum,Buchungsnummer,Kundennummer,MIETE,Hausnation,Name1,Name2,Adresse1,Adresse2,LKZ,...,Adresse1_repl,Adresse2_repl,Ort_repl,vorname_clean,nachname_clean,strasse_clean,hausnr_clean,ort_clean,plz_clean,newKey
0,2016-07-05,100003,3355207,1246,DK,Heiner Warzecha,NaN,Laubstr. 24,NaN,D,...,laubstr. 24,nan,bielefeld,heiner,warzecha,laubstr.,24,bielefeld,33607,warzecha laubstr.24 33607bielefeld
1,2016-07-05,100004,3788878,1506,DK,"Knöchelmann, Matthias",NaN,Bahnhofstr.48 A,NaN,D,...,bahnhofstr.48 a,nan,puls,matthias,knoechelmann,bahnhofstr.,48a,puls,25560,knoechelmann bahnhofstr.48a 25560puls
2,2016-07-05,100005,3788879,1656,DK,"Lippert, Yvonne",NaN,Eichenkamp 7,NaN,D,...,eichenkamp 7,nan,handorf,yvonne,lippert,eichenkp,7,handorf,21447,lippert eichenkp7 21447handorf
3,2016-07-05,100010,3665618,1966,D,"Lindenblatt, Merle",NaN,Wiehenweg 135,NaN,D,...,wiehenweg 135,nan,luebbecke,merle,lindenblatt,wiehenweg,135,luebbecke,32312,lindenblatt wiehenweg135 32312luebbecke
4,2016-07-05,100011,3665618,2886,D,"Lindenblatt, Merle",NaN,Wiehenweg 135,NaN,D,...,wiehenweg 135,nan,luebbecke,merle,lindenblatt,wiehenweg,135,luebbecke,32312,lindenblatt wiehenweg135 32312luebbecke


In [18]:
dat.to_pickle('buchungen_clean.pkl')

## Clean data and make same key in Versandadressen

### Import data

In [19]:
## Katalogversendungen
v1_fruehbucher = pd.read_csv('VersandadressenDM/Frühbucher.csv',sep=';')
v2_katalogbestellkarte = pd.read_csv('VersandadressenDM/Katalogbestellkarte.csv',sep=';')
v3_dtl=pd.read_csv('VersandadressenDM/Mailing Deutschland.csv',sep=';')
v4_lastMin = pd.read_csv('VersandadressenDM/Mailing Last Minute.csv',sep=';')
v5_nordsee = pd.read_csv('VersandadressenDM/Mailing Nordsee.csv',sep=';')
v6_norwegen = pd.read_csv('VersandadressenDM/Mailing Norwegen.csv',sep=';')
v7_ostsee = pd.read_csv('VersandadressenDM/Mailing Ostsee.csv',sep=';')
v8_schweden = pd.read_csv('VersandadressenDM/Mailing Schweden.csv', sep=';')

# new column names
v1_fruehbucher.columns =['quelle','mediacode','name1','name2','adresse1','plz','ort','nix']
v2_katalogbestellkarte.columns =['quelle','mediacode','kundennr','name1','name2','name3',
                                 'adresse1','plz','ort']
v3_dtl.columns =['quelle','mediacode','name1','name2','adresse1','plz','ort','adresse2']
v4_lastMin.columns =['quelle','mediacode','name1','adresse1','adresse2','plz','ort']
v5_nordsee.columns =['quelle','mediacode','kundennr','name1','name2','name3',
                     'adresse1','plz','ort','adresse2']
v6_norwegen.columns =['quelle','mediacode','kundennr','name1','name2','name3','name4',
                      'name5','name6','name7','adresse1','plz','ort']
v7_ostsee.columns =['quelle','mediacode','kundennr','name1','name2','name3',
                    'adresse1','plz','ort','adresse2']
v8_schweden.columns =['quelle','mediacode','kundennr','name1','adresse1','plz','ort']

In [20]:
## Erstversand
e1_daenemark = pd.read_csv('VersandadressenErstversand/Versand Dänemark.csv', sep=';',
                           dtype={'PLZ':str})
e2_norwegen = pd.read_csv('VersandadressenErstversand/Versand Norwegen.csv', sep=';')
e3_rbKunden = pd.read_csv('VersandadressenErstversand/Versand RB Kunden.csv',sep=';')
e4_schweden = pd.read_csv('VersandadressenErstversand/Versand Schweden.csv',sep=';',)
e5_deutschland = pd.read_csv('VersandadressenErstversand/Versand Deutschland.csv',sep=';')

# renaming column names
e1_daenemark.columns = ['quelle', 'mediacode','kundennr','name1','name2','adresse1',
                        'adresse2','plz','ort']
e2_norwegen.columns = ['quelle', 'mediacode','kundennr','name1','name2','adresse1',
                        'adresse2','plz','ort']
e3_rbKunden.columns = ['quelle', 'mediacode','kundennr','name1','name2','adresse1',
                        'adresse2','plz','ort','agnr','nix']
e4_schweden.columns = ['quelle', 'mediacode','kundennr','name1','name2','adresse1',
                        'adresse2','plz','ort']
e5_deutschland.columns = ['quelle', 'mediacode','kundennr','name1','name2','adresse1',
                        'adresse2','plz','ort','nix1','nix2','nix3']

In [21]:
data_list =  {'v1_fruehbucher':v1_fruehbucher,
             'v2_katalogbestellkarte':v2_katalogbestellkarte,
             'v3_dtl':v3_dtl,
             'v4_lastMin':v4_lastMin,
             'v5_nordsee':v5_nordsee,
             'v6_norwegen':v6_norwegen,
             'v7_ostsee':v7_ostsee,
             'v8_schweden':v8_schweden,
             'e1_daenemark':e1_daenemark,
             'e2_norwegen':e2_norwegen,
             'e3_rbKunden':e3_rbKunden,
             'e4_schweden':e4_schweden,
              'e5_deutschland':e5_deutschland,
             }
            

# function for cleaning versandadressen
def clean_versandadressen(dat):
    dat['name1_clean'] = dat['name1'].apply(clean_string)
    dat['nachname_clean'] = dat['name1_clean'].apply(name_handling_lastname)
    
    dat['adresse1_clean'] = dat['adresse1'].apply(clean_string)
    dat['adresse1_clean'] = dat['adresse1_clean'].apply(clean_streetnames_loaction)
    dat['strasse_clean'] = dat['adresse1_clean'].apply(delete_housenumber)
    dat.strasse_clean = dat.strasse_clean.apply(delete_spaces)
    dat['hausnr_clean'] = dat['adresse1_clean'].apply(extract_housenumber)
    
    dat['plz_clean'] = dat['plz'].apply(delete_leading_zeros)
    
    dat['ort_clean'] = dat['ort'].apply(clean_string)
    dat['ort_clean'] = dat['ort_clean'].apply(clean_streetnames_loaction)
    dat.ort_clean = dat.ort_clean.apply(delete_spaces)
    
    dat['newKey'] = dat['nachname_clean']+' '+dat['strasse_clean']+ \
                    dat['hausnr_clean']+' '+dat['plz_clean']+dat['ort_clean']
    
    return dat


## apply function to all tables of versandadressen
versandadressen_clean = {}
for key, dat in data_list.items():
    versandadressen_clean[key] = clean_versandadressen(dat)
    
## save versandadressen

import pickle
pickle.dump(versandadressen_clean,open('versandadressen_clean.pkl','wb'))

magdeburg universitaet sportcentrum
aussenwohngruppe
globalforwarding g. gmbh
schottener soziale diens
schumann tank u. stahlbau gmbh
lebenshilfe wolfsburg
lindner brasausky gmbh
evangelische jugendhilfe
institut fuer quantenop.
diakonische wohnheime himmelsthuer ggmbh
st. liborius kirchengemeinde
elektro staif gmbh
eg.wohnhilfen rendsburg
kw-montageservice
muerwiker werkstaetten gmbh
igefa weinbrenner waermetausch gmbh
spastikerhilfe berlin e.g
bundesanstalt thw jugengruppe
evg-luth. kirchengemeinde sittensen
trebbiner kinder- u. jugendheim e.v.
paritaetischer hh e.v.
martinsclub bremen
paritaetische lebenshilfe
stephanus stiftung
fussball foerderverein wingst
stiftung lebensraeume ovelgoenner muehle
bpd gmbh
ingmont berlin gmbh
infra bauservice gmbh
sosa brothers gmbh
zusammenleben e.v. ulrike benkart
wicon gmbh gerd trappmann
green gmbh arne green
assmann gmbh assmann gmbh
jugendhilfe netzwerk nor
martin delz hof delz e.v.
riis construction gmbh inga lorenzen
hof althing gem. gmbh h

institut fuer quantenop.
stephansstift wohngruppe gihon
schweerbau gmbh u. co. kg krause
moeller feeurfesttechnik gmbh u. co.kg
hekabe design gmbh
bdks hofgeismar gruppe 3, frau landau
jugendhilfe hephata johannes janke
rovema gmbh
guenter meier innenausbau gmbh
mansfeld-loebbecke-stiftung
magdeburg universitaet sportcentrum
firma minimax e.i.s. gmbh letzien
schumann tank u. stahlbau gmbh
schumann tank u. stahlbau gmbh
holz tec innenausbau gmbh
phereclus blade services
ev. stiftung hephata wohnen ggbmh
stiftung hephata wohnen ggmbh
oez automobile gmbh
maedchenwohngruppe
eon anlagenservice
lebenshilfe dinslaken hans-joachim meyer
heilpaedagogische hilfe
elrond niedersachsen gem. e. v.
kw-montageservice
ev.wohngruppe diepholz
firma vwa-deutschland gmbh
gregory bedachungen gmbh
bueter+jeurink gmbh
xstone design gmbh
dmi industrieservice gmb
igefa weinbrenner waermetausch gmbh
b.droessler gmbh u. co.
srs e.v. andrea porsch
meyer service u. logistik
bundesanstalt thw jugendgruppe
kinetics g

jc eckardt gmbh
eon anlagenservice
ese gmbh
lehmann bau berlin gmbh
hof neumuehlen gmbh herr stefan werner
minimax e.i.s. gmbh
loehrke, juergen gmbh
imar gmbh
stephanus stiftung
thomas fischer haus, gruppe 1
stiftung bethel kostenstelle 1021120
martinsclub bremen
lebenshilfe bremervoerde/zeven
lebenshilfe celle ggmbh wendt
diagnosegruppe emsdetten
viadico gmbh
muerwiker werkstaetten gmbh
matthew u. kirsten service
schottener soziale diens
zusammenleben e.v. aline schaak
paritaetische lebenshilfe
kinder- u. jugendhilfe st. joseph
triangelgruppe
diakonische wohnheime himmelsthuer ggmbh
koelner verein f. rehabilitation
stiftung eben-etzer
intensiv paedagogische kinderwohngruppe
datagroup c.s. gmbh
shipshape deutschland gmbh h. eberhardt
alexiana muenster gmbh
aussenwohngruppe
eg.wohnhilfen rendsburg
lebenshilfe stade e.v.
trebbiner kinder- u. jugendheim e.v.
ev. stiftung volmarstein hr. dombrowsky
thomas niess fa. viadico gmbh
